In [97]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Reshape, Attention, TimeDistributed, Embedding, LSTM, Bidirectional, Dropout, Dense, GRU, concatenate, Conv1D, Conv2D, Flatten,MaxPooling1D, MaxPooling2D, LocallyConnected1D, Activation, GaussianNoise, BatchNormalization
from tensorflow.keras.models import Model
import nltk
from AttentionWeightedAverage import AttentionWeightedAverage
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

- NLP on Title + Description to Tags, in order to build a tag recommendation algorithm
- NLP on Tags + Description + Category to Title, in order to provide title recommendation algorithm. These videos are popular videos, meaning they have appealing titles. Let's build a system, that judging by content, can provide the most appropriate title.

In [2]:
df = pd.read_csv("./data/USvideos.csv")
print('Number of youtube videos in dataframe: ', len(df.index))
df.head()

# count category occurences
occurences = df['category_id'].value_counts()
# category names
map_categories = {}
with open('data/US_category_id.json') as json_file:
    data_category = json.load(json_file)
    for category in data_category['items']:
        int_id = int(category['id'])
        occ = 0
        if int_id in occurences:
            occ = occurences[int_id]
        map_categories[int_id] = [category['snippet']['title'],occ]
# create new dataframe with categories and appearances
df_categories = pd.DataFrame.from_dict(map_categories, orient='index', columns=['category_name', 'count'])
# add category name on the main dataframe
for i in map_categories.keys():
    df.loc[df['category_id'] == i,'category_name'] = map_categories[i][0]

Number of youtube videos in dataframe:  40949


### Tags, Description and Category -> Auto generate Title

In [3]:
def discretize(tags):
    tags = tags.replace('"','')
    discrete_tags = tags.split('|')
    return discrete_tags

In [4]:
df_text = df[['title', 'description', 'tags', 'category_name']]
df_text = df_text.dropna(axis=0, how='any')

titles = df_text['title'].tolist()
descriptions = df_text['description'].tolist()
tags = [discretize(row['tags']) for index, row in df_text.iterrows()]
categories = df_text['category_name'].tolist()

- Titles need to be tokenized, lowercased, categorical.
- Descriptions need to be tokenized, lowercased, categorical.
- Tags need to be lowercased, categorical.

In [5]:
clean_desc = []
clean_title = []
for desc in descriptions:
    desc = desc.replace('\\n', ' ')
    desc = desc.lower()
    desc = desc.replace('www.facebook.com', 'FacebookLink')
    desc = desc.replace('http.facebook.com', 'FacebookLink')
    desc = desc.replace('www.twitter.com', 'TwitterLink')
    desc = desc.replace('www.amazon.com', 'AmazonLink')
    desc = desc.replace('http.amzn.com', 'AmazonLink')
    desc = desc.replace('www.instagram.com', 'InstagramLink')
    desc = desc.replace('www.snapchat.com', 'SnapchatLink')
    desc = desc.replace('https://bit.ly', 'PortalLink')
    clean_desc.append(text_to_word_sequence(desc, filters='!"#%&()*-+,.\\/:;<=>?@[\\]^_`{|}~\t\n♡▶➜', lower=True, split=' '))
    
for title in titles:
    title = title.replace('\\n', ' ')
    seq = text_to_word_sequence(title, filters='!"#%&()*-+,.\\/:;<=>?@[\\]^_`{|}~\t\n♡▶➜', lower=True, split=' ')
    token_seq = []
    for w in seq:
        token_seq += list(w)
        token_seq += " "
    clean_title.append(token_seq)

Training an unsupervised method for word2vec with gensim, in order to provide me with very good embeddings

In [6]:
from gensim.models import Word2Vec

gensim_model = Word2Vec(size=100,
                       window=5,
                       min_count=1,
                       sg=1,
                       hs=0,
                       negative=10,
                       workers=4)
gensim_model.build_vocab(clean_desc+clean_title+tags)
gensim_model.train(clean_desc+clean_title+tags, total_examples=len(clean_desc+clean_title+tags), epochs=10)

(70696706, 91275720)

Creating my new sequences with vector representations of words

In [8]:
X_train_description = []
X_train_category = []
X_train_tags = []
Y_train_title = []

lb_cat = LabelBinarizer()
lb_cat.fit(list(set(categories)))

maxLenDesc = 0
for desc in clean_desc:
    X_train_description.append(np.asarray([gensim_model.wv[word] for word in desc]))
    if len(desc) > maxLenDesc:
        maxLenDesc = len(desc)
        
maxLenTags = 0
for taglist in tags:
    X_train_tags.append(np.asarray([gensim_model.wv[tag] for tag in taglist]))
    if len(taglist) > maxLenTags:
        maxLenTags = len(taglist)

for cat in categories:
    X_train_category.append(lb_cat.transform([cat]))

vocab_chars = {}
maxLenTitle = 0
vocab_char_size = 0
for title in clean_title:
    for ch in title:
        if ch not in vocab_chars.keys():
            vocab_chars[ch] = vocab_char_size
            vocab_char_size += 1

In [12]:
lb_title = LabelBinarizer()
one_hots = lb_title.fit_transform(range(len(vocab_chars.keys())))

Y_train_title = []
maxLenTitle = 0
for title in clean_title:
    Y_train_title.append(np.asarray([one_hots[vocab_chars[ch]] for ch in title]))
    if len(title) > maxLenTitle:
        maxLenTitle = len(title)

In [13]:
# padding
X_train_description = pad_sequences(X_train_description, maxlen=maxLenDesc, padding='post')
X_train_tags = pad_sequences(X_train_tags, maxlen=maxLenTags, padding='post')
Y_train_title = pad_sequences(Y_train_title, maxlen=maxLenTitle, padding='post', value=vocab_char_size+1)

# convert to numpy arrays
X_train_description = np.stack(X_train_description, axis=0)
X_train_category = np.stack(X_train_category, axis=0)
X_train_tags = np.stack(X_train_tags, axis=0)
Y_train_title = np.stack(Y_train_title, axis=0)

#### Gen RNN Model

In [154]:
def text_genrnn(char_vocab_size,maxLenDesc,maxLenTitle,input_features,batch_size=101):
    embedding_desc = Input(batch_input_shape=(None,None,input_features), name='main_input')
    
    rnn_1, state_11, state_12= LSTM(128, return_sequences=True, return_state=True, name='lstm1')(embedding_desc)
    rnn_2, state_21, state_22 = LSTM(128, return_sequences=True, return_state=True, name='lstm2')(rnn_1)
    
    rnn_concat = concatenate([embedding_desc, rnn_1, rnn_2], axis=-1)
    state_concat = concatenate([state_11, state_21], axis=-1)
    
    attention = Attention(name='attention')([rnn_1, rnn_2])
    
    output = Dense(char_vocab_size, name='output', activation='softmax')(attention)
    
    model = Model(inputs=[embedding_desc], outputs=[output])
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=4e-3))
    return model

In [155]:
epochs=10
batch_size=32
model = text_genrnn(vocab_char_size,maxLenDesc,maxLenTitle,100)
model.summary()

Model: "model_29"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
lstm1 (LSTM)                    [(None, None, 128),  117248      main_input[0][0]                 
__________________________________________________________________________________________________
lstm2 (LSTM)                    [(None, None, 128),  131584      lstm1[0][0]                      
__________________________________________________________________________________________________
attention (Attention)           (None, None, 128)    0           lstm1[0][0]                      
                                                                 lstm2[0][0]               

In [ ]:
model.fit(X_train_description,Y_train_title, epochs=epochs, batch_size=batch_size, verbose=1)